<a href="https://colab.research.google.com/github/stebechoi/CP2/blob/main/4.%20(MovieLense100K)_CF_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MovieLense 데이터를 이용한 영화 추천 시스템 (CF_k-최근접)

## K-NearestNeighbors

[참고](https://www.youtube.com/watch?v=4Ws0oPH350U)

### 데이터 로드

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/CP/CP2')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('movielense_rt100.csv', usecols=['user_id', 'movie_id', 'movie_title', 'rating', 'TotalRatingCount'], dtype={'user_id':'int32', 'movie_id':'int32', 'movie_title':'str', 'rating':'int32'})
df

,user_id,movie_id,movie_title,rating,TotalRatingCount
0,308,1,Toy Story (1995),4,452
1,308,4,Get Shorty (1995),5,209
2,308,7,Twelve Monkeys (1995),4,392
3,308,8,Babe (1995),5,219
4,308,9,Dead Man Walking (1995),4,299
...,...,...,...,...,...
65003,729,346,Jackie Brown (1997),1,126
65004,729,690,Seven Years in Tibet (1997),2,155
65005,729,748,"Saint, The (1997)",4,316
65006,729,751,Tomorrow Never Dies (1997),3,180


In [4]:
df['TotalRatingCount'].min()

100

In [5]:
df.groupby(['movie_id'])['rating'].count().sort_values()

movie_id
865      21
500      31
297      50
305      87
1012    100
       ... 
294     485
181     507
100     508
258     509
50      583
Name: rating, Length: 342, dtype: int64

In [6]:
df_movie = df[['movie_id', 'movie_title', 'user_id', 'rating']].sort_values(['movie_id'], ascending=True)
df_movie['title'] = df_movie['movie_title']

In [7]:
df_movie=df_movie[['movie_id', 'title', 'user_id', 'rating']]
df_movie

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
6085,1,Toy Story (1995),141,3
35008,1,Toy Story (1995),542,4
34972,1,Toy Story (1995),779,4
34864,1,Toy Story (1995),882,5
...,...,...,...,...
13319,1047,Multiplicity (1996),291,2
881,1047,Multiplicity (1996),181,2
28740,1047,Multiplicity (1996),872,4
11605,1047,Multiplicity (1996),472,4


### 형태변환

In [8]:
# Sparse Matrix로 변환
from scipy.sparse import csr_matrix

In [9]:
# movies_users2 = df_movie.pivot_table(index='title', columns='user_id', values='rating').fillna(0) # nan이 결측값의 의미가 아님, 보지 않은 영화로 0으로 대체(점수범위는 (1,5))
# # Index contains duplicate entries, cannot reshape -> pivot 아닌 pivot_table
# # pivot일 때 movie_id는 동작, movie_title은 오류
# movies_users2

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
title,,,,,,,,,,,,,,,,,,,,,
101 Dalmatians (1996),2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),4.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
Absolute Power (1997),0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
"Abyss, The (1989)",3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willy Wonka and the Chocolate Factory (1971),4.0,0.0,0.0,0.0,3.0,3.0,4.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,4.0
"Wizard of Oz, The (1939)",4.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
"Wrong Trousers, The (1993)",5.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
movies_users = df.pivot(index='movie_id', columns='user_id', values='rating').fillna(0) # nan이 결측값의 의미가 아님, 보지 않은 영화로 0으로 대체(점수범위는 (1,5))
movies_users

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,4.0,0.0,0.0,0.0,0.0,2.0,5.0,3.0,4.0,4.0,...,0.0,0.0,4.0,0.0,4.0,0.0,4.0,4.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0
1012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [11]:
movies_users.index

Int64Index([   1,    2,    4,    7,    8,    9,   11,   12,   13,   14,
            ...
             865,  866,  879,  895,  926,  928, 1012, 1016, 1028, 1047],
           dtype='int64', name='movie_id', length=342)

In [20]:
# 인덱스 - movie_id - movie_title
movie_df = df.sort_values(['movie_id']).drop_duplicates(subset=['movie_id']).reset_index(drop=True)[['movie_id', 'movie_title']]
movie_df

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,4,Get Shorty (1995)
3,7,Twelve Monkeys (1995)
4,8,Babe (1995)
...,...,...
337,928,"Craft, The (1996)"
338,1012,Private Parts (1997)
339,1016,Con Air (1997)
340,1028,Grumpier Old Men (1995)


In [21]:
# 행렬변환
mat_movies_users = csr_matrix(movies_users.values)
mat_movies_users

<342x943 sparse matrix of type '<class 'numpy.float64'>'
	with 65008 stored elements in Compressed Sparse Row format>

In [14]:
# movies_users의 인덱스 = movie_id로 접근 불가.
# mat_movies_users는 건너띄기 없는 index

In [15]:
print(mat_movies_users)

  (0, 0)	5.0
  (0, 1)	4.0
  (0, 4)	4.0
  (0, 5)	4.0
  (0, 9)	4.0
  (0, 12)	3.0
  (0, 14)	1.0
  (0, 15)	5.0
  (0, 16)	4.0
  (0, 17)	5.0
  (0, 19)	3.0
  (0, 20)	5.0
  (0, 22)	5.0
  (0, 24)	5.0
  (0, 25)	3.0
  (0, 37)	5.0
  (0, 40)	4.0
  (0, 41)	5.0
  (0, 42)	5.0
  (0, 43)	4.0
  (0, 44)	5.0
  (0, 48)	2.0
  (0, 53)	4.0
  (0, 55)	4.0
  (0, 56)	5.0
  :	:
  (341, 748)	3.0
  (341, 757)	3.0
  (341, 789)	3.0
  (341, 791)	3.0
  (341, 799)	3.0
  (341, 803)	3.0
  (341, 824)	3.0
  (341, 836)	1.0
  (341, 850)	3.0
  (341, 853)	1.0
  (341, 859)	2.0
  (341, 863)	3.0
  (341, 864)	1.0
  (341, 868)	2.0
  (341, 871)	4.0
  (341, 878)	2.0
  (341, 879)	3.0
  (341, 886)	5.0
  (341, 900)	3.0
  (341, 906)	5.0
  (341, 918)	3.0
  (341, 920)	1.0
  (341, 926)	4.0
  (341, 937)	3.0
  (341, 942)	2.0


In [22]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute') # brute: 무차별 대입 접근

In [23]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine')

### 추천

In [24]:
# fuzzy matching 두개의 문자열 일치 시키기 (철자오류 공백 오타 등이 있어도 mashed하여 결과출력)
!pip install fuzzywuzzy
from fuzzywuzzy import process

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [77]:
# Recommender(movie_name) => List of Movies recommended

def recommender(movie_name, data, model, n_recommendations):
  model.fit(data)
  print('Working Now..')
  idx = process.extractOne(movie_name, movie_df['movie_title'])[2]  # recommender('toystory') # ('Toy Story (1995)', 79, 0) <- (일치하는 이름, 일치확률, 해당 인덱스): 그러니까 인덱스는 [2]위치에 있다.
  print('Movie Selected: ',movie_df['movie_title'][idx],  'Index: ',idx)
  print('Searching for recommendations....')

  movie_id = movie_df['movie_id'][idx]
  distances, indices = model.kneighbors(data[idx], n_neighbors = n_recommendations+1) #idx로 하나로 특정(동명이인 제외 하듯)
  print(distances, indices)
  for i in indices: 
    print(movie_df.iloc[i[1:]]) # itself: 본인 제외

In [79]:
# 예시
recommender('alone', mat_movies_users, model_knn, 4) 

Working Now..
Movie Selected:  Home Alone (1990) Index:  51
Searching for recommendations....
[[1.11022302e-16 3.13485691e-01 3.71861180e-01 4.08788066e-01
  4.24615707e-01]] [[ 51 229  32 231  42]]
     movie_id                        movie_title
229       393              Mrs. Doubtfire (1993)
32         67  Ace Ventura: Pet Detective (1994)
231       403                      Batman (1989)
42         82               Jurassic Park (1993)


### 평가

In [ ]:
# 추천 시스템에서의 k-NN은 비지도 학습?
# 평가지표?